In [13]:
import os
import xml.etree.ElementTree as ET
import spacy
import pandas as pd
from sklearn.metrics import classification_report
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
import json
import xml.etree.ElementTree as ET
import re

In [28]:
# Chemins vers les dossiers
BASE_DIR = "data/PL"
MANUELLES_DIR = os.path.join(BASE_DIR, "manuelles")
AUTOMATIQUES_DIR = os.path.join(BASE_DIR, "automatiques")

In [29]:
# Ensemble pour stocker les mots uniques
mots_uniques = set()

# Parcourir tous les fichiers .eaf dans le dossier
for nom_fichier in os.listdir(MANUELLES_DIR):
    if nom_fichier.endswith(".eaf"):
        chemin_fichier = os.path.join(MANUELLES_DIR, nom_fichier)
        arbre = ET.parse(chemin_fichier)
        racine = arbre.getroot()

        # Trouver le tier avec l'ID "%pol"
        for tier in racine.findall(".//TIER[@TIER_ID='%pol']"):
            for annotation in tier.findall(".//ALIGNABLE_ANNOTATION/ANNOTATION_VALUE"):
                texte = annotation.text.strip().lower()  # Nettoyer et mettre en minuscules
                mots = texte.split()  # Diviser le texte en mots (selon les espaces)
                mots_uniques.update(mots)  # Ajouter les mots à l'ensemble (pas de doublons)

# Convertir l'ensemble en liste et trier alphabétiquement
liste_mots = sorted(mots_uniques)

# Afficher la liste des mots uniques
print(liste_mots)
len(liste_mots)

['a', 'adama', 'budynek', 'być', 'będzie', 'cały', 'całą', 'chciał', 'curie', 'czas', 'cztery', 'czwarty', 'dalej', 'dobra', 'dobrze', 'dobrą', 'dojść', 'dokładnie', 'domek', 'gdzie', 'i', 'idzie', 'iść', 'jest', 'juliusza', 'już', 'ją', 'kierować', 'kolejowej', 'lewej', 'lewo', 'marii', 'mickiewicza', 'mija', 'minąć', 'międzyczasie', 'musi', 'na', 'nadal', 'najbliższe', 'najpierw', 'należy', 'następnie', 'niską', 'nią', 'no', 'numer', 'pan', 'pasy', 'pewno', 'pierwszym', 'po', 'potem', 'powinien', 'powinieneś', 'prawo', 'prosto', 'proszę', 'przejść', 'przez', 'przy', 'pójść', 'rozpozna', 'się', 'sklep', 'sklepem', 'skrzyżowanie', 'skrzyżowaniu', 'skręca', 'skręcić', 'skłodowskiej', 'stacji', 'stronie', 'szpitalu', 'szybko', 'słowackiego', 'tak', 'taki', 'tam', 'teatr', 'tej', 'teraz', 'też', 'to', 'trzeba', 'ulica', 'ulicy', 'ulicą', 'ulicę', 'w', 'wyjść', 'z', 'za', 'zaraz', 'ze', 'zielony']


96

In [30]:
# Charger le modèle spaCy pour le polonais
nlp = spacy.load("pl_core_news_sm")

In [31]:
donnees = []

for mot in liste_mots:
    doc = nlp(mot)
    for token in doc:
        # Garder seulement les noms (NOUN) et adjectifs (ADJ)
        if token.pos_ in ["NOUN", "ADJ"]:
            # Récupérer le cas grammatical si disponible
            cas = token.morph.get("Case")
            donnees.append({
                "mot": token.text,
                "cas": cas[0] if cas else "-"
            })

# Créer un tableau avec pandas
df_spacy = pd.DataFrame(donnees)

In [32]:
# Sauvegarder dans un fichier CSV
df_spacy.to_csv("corpus_polonais_cas_spacy.csv", index=False, encoding="utf-8")

In [33]:
# Langues cibles à détecter
langues_cibles = {"fr", "it", "nl", "en", "de"}

# Dictionnaire pour stocker les mots détectés par langue
mots_par_langue = {code: set() for code in langues_cibles}

# Détection de la langue pour chaque mot unique
for mot in liste_mots:
    try:
        if len(mot) < 3:
            continue  # Ignorer les mots trop courts (peu fiables)
        langue = detect(mot)  # Détecter la langue du mot
        if langue in langues_cibles:
            mots_par_langue[langue].add(mot)  # Ajouter au bon groupe
    except LangDetectException:
        continue  # Passer les cas non détectables (ex. mot vide)

# Convertir les ensembles en listes triées
mots_par_langue_final = {
    lang: sorted(list(mots)) for lang, mots in mots_par_langue.items() if mots
}

# Affichage du dictionnaire final (comme demandé)
print(mots_par_langue_final)


{'it': ['teraz']}


In [34]:
# Chargement du dossier corrigé
df_corrige = pd.read_csv('corpus_polonais_cas_corrigé.csv')

In [35]:
# Comparaison des cas
y_pred = df_spacy['cas'].fillna('UNK')
y_true = df_corrige['cas'].fillna('UNK')

# Rapport de classification
report = classification_report(y_true, y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         Acc       1.00      0.50      0.67         2
         Dat       0.00      0.00      0.00         0
         Gen       0.25      0.67      0.36         3
         Ins       0.86      1.00      0.92         6
         Loc       0.00      0.00      0.00         7
         Nom       0.79      0.94      0.86        16
         UNK       0.00      0.00      0.00         2

    accuracy                           0.67        36
   macro avg       0.41      0.44      0.40        36
weighted avg       0.57      0.67      0.60        36



In [36]:
# Précision
mask_predicted = df_spacy['cas'] != ''
true_positives = (df_spacy['cas'] == df_corrige['cas']) & mask_predicted

precision_spacy = true_positives.sum() / mask_predicted.sum()

print(f"Précision de spaCy sur l'attribution des cas : {precision_spacy * 100:.2f}%")

Précision de spaCy sur l'attribution des cas : 66.67%


In [40]:
# Fonction pour nettoyer la transcription automatique
def clean_text(text):
    # supprimer la ponctuation
    text = re.sub(r"[.,!?;:]", "", text)
    # mettre tout en minuscules
    text = text.lower()
    # supprimer les espaces et retours multiples
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Fonction pour extraire la transcription manuelle d’un fichier .eaf
def extract_manual_transcription(eaf_file):
    tree = ET.parse(eaf_file)
    root = tree.getroot()
    manual_segments = []
    # chercher la tier "%pol" et récupérer toutes les annotations
    for tier in root.findall(".//TIER[@TIER_ID='%pol']"):
        for ann in tier.findall("ANNOTATION/ALIGNABLE_ANNOTATION/ANNOTATION_VALUE"):
            manual_segments.append(ann.text.strip())
    # concaténer les segments en un seul texte
    return " ".join(manual_segments)

# Fonction pour extraire et nettoyer la transcription automatique d’un fichier .txt
def extract_auto_transcription(txt_file):
    with open(txt_file, "r", encoding="utf-8") as f:
        content = f.read()
    return clean_text(content)

def main():
    corpus = []

    # parcourir les fichiers .eaf dans le dossier manuelles
    for filename in os.listdir(MANUELLES_DIR):
        if filename.endswith(".eaf"):
            name = os.path.splitext(filename)[0]  # ex. "Klaudia"
            eaf_path = os.path.join(MANUELLES_DIR, filename)
            txt_path = os.path.join(AUTOMATIQUES_DIR, f"{name}.txt")

            # vérifier si le fichier automatique correspondant existe
            if not os.path.exists(txt_path):
                print(f"Fichier automatique manquant pour {name}")
                continue

            # extraire les deux transcriptions
            manual = extract_manual_transcription(eaf_path)
            auto = extract_auto_transcription(txt_path)

            # ajouter un exemple au corpus
            corpus.append({
                "name": name,
                "transcription manuelle": manual,
                "transcription automatique": auto
            })

    with open("corpus_polonais_natif.json", "w", encoding="utf-8") as f:
        json.dump(corpus, f, ensure_ascii=False, indent=4)

    print(f"Corpus enregistré.")

if __name__ == "__main__":
    main()

Corpus enregistré.
